In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

In [6]:
data = pd.read_csv('./data/preprocessed/full_data.1.csv', index_col='Id')

In [7]:
data.head()

,Body,Tags,Title,tag_javascript,tag_python,tag_java,tag_android,tag_php,tag_c#,tag_html,...,tag_postgresql,tag_perl,tag_sqlite,tag_shell,tag_machine-learning,tag_csv,tag_datetime,tag_multithreading,tag_object,tag_android-layout
Id,,,,,,,,,,,,,,,,,,,,,
34862324,Algorithmm input Graph G output Set of ...,java algorithm graph-algorithm,I am implementing a Minimum Spanning Forrest a...,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
47303682,While converting the data frame to HTML Date i...,html r dataframe,R Studio Date is getting converted to number w...,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
46431243,This should be an easy one but somehow I could...,python pandas dataframe pandas-groupby,Pandas dataframe groupby how to get sum of m...,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
45306324,Let say I have a component in Angular 2 that n...,angular typescript rxjs,Casting results from ObservableforkJoin to the...,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
49479547,Im setting a var using set TESTVAR5 and then...,c windows batch-file,Environment variable set in batch file cannot ...,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [10]:
pd.isna(data[['Body', 'Tags', 'Title']]).sum()

Body     0
Tags     0
Title    2
dtype: int64

In [11]:
data[data['Title'].isna()]

,Body,Tags,Title,tag_javascript,tag_python,tag_java,tag_android,tag_php,tag_c#,tag_html,...,tag_postgresql,tag_perl,tag_sqlite,tag_shell,tag_machine-learning,tag_csv,tag_datetime,tag_multithreading,tag_object,tag_android-layout
Id,,,,,,,,,,,,,,,,,,,,,
42407720,npm WARN optional SKIPPING OPTIONAL DEPENDENCY...,node.js mongodb,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36240495,I trying to make a request GET to import req...,python,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
data['Title'].fillna('', inplace=True)

In [14]:
pd.isna(data[['Body', 'Tags', 'Title']]).sum()

Body     0
Tags     0
Title    0
dtype: int64

# BoW

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
x_train = data[['Body', 'Tags', 'Title']]
data = data.drop(['Body', 'Tags', 'Title'], axis=1)

In [28]:
x_bow_body = CountVectorizer(ngram_range=(1, 2), max_features=100, dtype=np.float32).fit_transform(x_train['Body'])

In [29]:
x_bow_tags = CountVectorizer(ngram_range=(1, 2), max_features=100, dtype=np.float32).fit_transform(x_train['Tags'])

In [30]:
x_bow_title = CountVectorizer(ngram_range=(1, 2), max_features=100, dtype=np.float32).fit_transform(x_train['Title'])

In [31]:
print(x_bow_body.shape)
print(x_bow_tags.shape)
print(x_bow_title.shape)

(60000, 100)
(60000, 100)
(60000, 100)


In [39]:
features_names = lambda prefix: [f'{prefix}_{i}' for i in range(100)]
bow_data = pd.concat([data,
                      pd.DataFrame(x_bow_body.toarray(), index=data.index, columns=features_names('body')),
                      pd.DataFrame(x_bow_tags.toarray(), index=data.index, columns=features_names('tags')),
                      pd.DataFrame(x_bow_title.toarray(), index=data.index, columns=features_names('title'))])
bow_data.to_csv('./data/preprocessed/bow_data.csv', index_label='Id')

# TF-IDF

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
x_tfidf_body = TfidfVectorizer(ngram_range=(1, 2), max_features=100, dtype=np.float32).fit_transform(x_train['Body'])

In [33]:
x_tfidf_tags = TfidfVectorizer(ngram_range=(1, 2), max_features=100, dtype=np.float32).fit_transform(x_train['Tags'])

In [34]:
x_tfidf_title = TfidfVectorizer(ngram_range=(1, 2), max_features=100, dtype=np.float32).fit_transform(x_train['Title'])


In [35]:
print(x_tfidf_body.shape)
print(x_tfidf_tags.shape)
print(x_tfidf_title.shape)

(60000, 100)
(60000, 100)
(60000, 100)


In [40]:
bow_data = pd.concat([data,
                      pd.DataFrame(x_tfidf_body.toarray(), index=data.index, columns=features_names('body')),
                      pd.DataFrame(x_tfidf_tags.toarray(), index=data.index, columns=features_names('tags')),
                      pd.DataFrame(x_tfidf_title.toarray(), index=data.index, columns=features_names('title'))])
bow_data.to_csv('./data/preprocessed/tfidf_data.csv', index_label='Id')

# Word2Vec

In [42]:
from gensim import downloader

list(downloader.info()['models'].keys())

['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']

In [43]:
w2v_model = downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [45]:
w2v_model.get_vector('aer 12ponveorpniv')

KeyError: "Key 'aer 12ponveorpniv' not present"

In [44]:
w2v_data = x_train.copy()
for column in ['Body', 'Tags', 'Title']:
    w2v_data[column].apply(lambda x: w2v_model[x])

KeyError: "Key 'Algorithmm    input  Graph G    output Set of MSTs T    begin    Tnull  EGEdges    for all vertices in G  Create a tree t having single vertex b    add t to T    end for         repeat          Find an edge e ∈ E having minimum weight          such that one end belongs to t ∈ T and the other          end does not belongs to any of the trees in T          Add e to t        until e  NULL    Im stuck on the logic for the highlighted block  Ive used simple objects for vertexedge and tree And for their sets used array of Objects      ' not present"